# Let the Machine Help Understand the Human Responses

## Bingqing Wang, 913473517

## Final Project STA 208


Social scientific studies often times involve human coders to read open-ended responses and code each response with numbers, so that follow-up statistical analysis can be employed. This type of method usually suffers several disadvantages.
    
  1. Human coders can suffer fatigue and inrealiability issues, which may result in unreliable coding for further analysis.
  2. Human coders are usually cost inefficient and are harder to find compared to computers. 
    
The present project aims at using machine learning technique to code textual responses. With minimal input and guidance, a machine can code the textual responses within seconds and with no fatigue and high reliability. The entire project consists of three part and two datasets I have collected in previous studies. 
    
First, I will employ the first dataset to test the LDA (Latent Dirichlet Allocation) method in abstracting topics from open-ended responses. The first step is to ensure that meaningful different topics can be recognized by the machine and there are distinctive different topics in the dataset. This is an insurance policy to make sure that scholars are makign meaningful inference and usage with the programming.
    
Second, I will use the second dataset to let machines do human coding. I will use existed topics summaires to guide the program do the coding. These topics summaries were usually easily available when people train human coders to code texts. They can be definitions of a concept, category distinctions, as well as coding rubrics. In this step, I will have a basic coding program to do human coding jobs.
    
Third, I will improve the program so that it can fit various demands in social scientific studies. Due to the complexity of human behavious, social scientific studies usually have different specific demands in differnt areas. For example, people may declare several meanings in one responses. The last step is to improve the program to adapt to different environment.

### Part I. Topic Modeling with LDA

In the first part, LDA method is enployed to abstract topics from participants' responses. The dataset is from a study I have done about video game playing in families and the open-ended question asked the participants:" Based on your expeiriences, what changes does video game bring to your families." I employed human coders to code these responses into three catrgories: social hedonic and eudaimonic fucnitons. The program below will try to extract these three topics from the responses.

In [1]:
import pandas as pd
import numpy as np

corp1 = pd.read_csv("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus1.csv")
corp1.head(5)

,"Based on your experiences, what changes does video game bring to your families?"
0,it gives us a chance to spend time together at...
1,"It brings a greater sense of bonding, and we c..."
2,nothing. just something to do. not different...
3,we can get together and play and have fun and ...
4,it makes us be more competitive with each othe...


In [2]:
corp1.describe()

,"Based on your experiences, what changes does video game bring to your families?"
count,364
unique,357
top,In leisure time we spend by playing games and ...
freq,6


In [3]:
corp1 = corp1.drop_duplicates(keep="first")
# complete documents
doc_complete = corp1
corp1.columns = ["EXP"]

corp1.head()

,EXP
0,it gives us a chance to spend time together at...
1,"It brings a greater sense of bonding, and we c..."
2,nothing. just something to do. not different...
3,we can get together and play and have fun and ...
4,it makes us be more competitive with each othe...


In [4]:
doc_complete = doc_complete.values.tolist()

In [5]:
doc_complete = [str(i) for i in doc_complete]

In [6]:
# understand the corpus
# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(str(corp1))
# convert to lower case
tokens = [w.lower() for w in tokens]
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:100])

['exp', 'gives', 'us', 'chance', 'spend', 'time', 'together', 'brings', 'greater', 'sense', 'bonding', 'c', 'nothing', 'something', 'different', 'get', 'together', 'play', 'fun', 'makes', 'us', 'competitive', 'othe', 'let', 'us', 'competitive', 'together', 'helps', 'something', 'everyone', 'enjoys', 'brings', 'us', 'makes', 'us', 'closer', 'together', 'commonality', 'fun', 'gives', 'us', 'something', 'sort', 'excitement', 'whille', 'playing', 'video', 'brings', 'sort', 'relief', 'happiness', 'brings', 'us', 'closer', 'together', 'able', 'ta', 'fun', 'activity', 'alll', 'agree', 'give', 'us', 'activity', 'enjoy', 'together', 'something', 'together', 'also', 'shoo', 'helps', 'us', 'bond', 'together', 'fun', 'acti', 'happy', 'satisfaction', 'children', 'gives', 'us', 'something', 'together', 'c', 'son', 'loves', 'play', 'gets', 'kick', 'feel', 'refreshed', 'relaxed', 'good', 'way', 'nt', 'really', 'change', 'anything', 'one', 'thing']


In [7]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]

In [8]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

C:\Users\jagwa\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, '0.056*"u" + 0.046*"it" + 0.042*"together"'), (1, '0.031*"game" + 0.018*"video" + 0.015*"time"'), (2, '0.034*"game" + 0.022*"video" + 0.018*"play"')]


### Part II. Improve the topic modeling

In this part, I will try to improve the topic modeling results using various strategies. Mainly, the goal is to find a optimal number of topics in a specific corpus. In the previous part, I suspect there are three topics in the corpus, but the number is by no means determined. In traditional social science studies, there are usually two ways of determining the number of topics researchers would like to code. First, reserchers can look up the theory and see how many topics theories suggest. Second, people can look at first few responses and have a brief understanding about how many topics there are in the corpus. Both of these techniques have serious disadvangtages. Theories and practices can be far away from each other and using theory to guide the coding does little in improving the theory. The first few responses can be vary biased and far from accuracy. Thus, using machining learing to help determine the number of topics can be a very beneficial tool in social science studies. 

I will use the same dataset from the previous part.

In [10]:
import spacy

# spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [11]:
# find the meanings of words, synonyms, antonyms, and more. In addition, we use WordNetLemmatizer to get the root word.

import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [12]:
# filter stop words
# nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [13]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [14]:
import random
text_data = []
with open("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus1.csv") as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['allow', 'together', 'develop', 'experience']
[]
[]
['think', 'bring', 'cooperation', 'communication', 'sense', 'accomplishment', 'depend', 'increase', 'daughter', 'dexterity', 'help', 'teach', 'solve', 'problem']
['together', 'something', 'enjoy', 'change', 'positive']
[]
[]
[]
[]
['help', 'inside', 'joke', 'saying', 'game', 'keep', 'connect', 'interest']
['helping', 'improve', 'decision', 'making', 'power']


In [15]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [16]:
import gensim
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.053*"help" + 0.053*"think" + 0.053*"cooperation" + 0.053*"dexterity"')
(1, '0.055*"saying" + 0.055*"keep" + 0.055*"connect" + 0.055*"game"')
(2, '0.115*"together" + 0.066*"something" + 0.066*"change" + 0.066*"positive"')


Next, we will use the results to code a new statement into one of the three categories.

In [17]:
new_doc = 'I like video games that make us bonding together.'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(3, 1), (23, 1)]
[(0, 0.11222827), (1, 0.42873764), (2, 0.45903412)]


In [18]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.082*"decision" + 0.082*"making" + 0.082*"helping" + 0.082*"power"')
(1, '0.082*"together" + 0.047*"interest" + 0.047*"saying" + 0.047*"connect"')
(2, '0.053*"help" + 0.053*"daughter" + 0.053*"cooperation" + 0.053*"accomplishment"')


In [19]:
# Visualizing 3 topics

dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model3.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\jagwa\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


From the graph above, it can be seen that topic one, whatever that topic is, is pretty insignificant in the corpus. This suggest that three topics do not fit the reponses, even though in theory, there should be three topics in the responses. Next, i will try two topics and see the results.

In [20]:
import gensim
NUM_TOPICS = 2
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model2.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"help" + 0.038*"dexterity" + 0.038*"think" + 0.038*"accomplishment"')
(1, '0.052*"together" + 0.050*"help" + 0.050*"joke" + 0.050*"game"')


In [21]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 2, id2word=dictionary, passes=15)
ldamodel.save('model2.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.058*"help" + 0.035*"bring" + 0.035*"teach" + 0.035*"dexterity" + 0.035*"increase"')
(1, '0.059*"together" + 0.055*"decision" + 0.055*"power" + 0.055*"helping" + 0.055*"positive"')


In [22]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model2.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\jagwa\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


From the interactive graph above, it can be seen that the two topics are about the same size, and although ther are a lot of overlaps betwee nthe two topics, some distinctions can be made to distinguish the two topics.

_Analysis of the first corpus_  
From the results above, it can be seen that there are two main topics in the resopnses. First, people enjoy playing video games and believe it is "something" in the families. Second, from the responses, it is clear that video games have the bonding effect in family settings, family members share activities and feel something in "common". Therefore, we can interpret the responses and answer the question "what do video games bring to families" like the following:  
Video games mainly have two functions in the family settings. First, it provides hedonism/fun to family members so that they have something to enjoy. Second, video games have social/relatedness fucntion to families, so that family members have something in common and they are be bonded together. 

### Part III. Applying the same technique to another dataset  
In part III, I will apply the same technique to another dataset that is newly collected. The new corpus comes from another study that investigates people's experience in consuming movies. The question asks the participants: Based on your experience, what does watching movies bring to your life. The question is open-ended, and there is no previous human coding on this corpus. In this part, I will apply the same technique and see how many topics I can extract from the responses. 

In [40]:
corp2 = pd.read_csv("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus2.csv")
corp2.head(5)

<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<input>:1: DeprecationWarning: invalid escape sequence \S
<ipython-input-40-91f68b74ffd2>:1: DeprecationWarning: invalid escape sequence \S
  corp2 = pd.read_csv("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus2.csv")


,Movie experience
0,It's a nice way to get away from real life and...
1,movie are based on the my life and very modera...
2,Entertainment. Diversion. A distraction from d...
3,it brings enjoyment and give me a view on how ...
4,It arouses my imagination. It depends on what ...


In [42]:
corp2.describe()

,Movie experience
count,359
unique,352
top,The best movies make you think while entertain...
freq,3


In [34]:
text_data_2 = []
with open("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus2.csv") as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data_2.append(tokens)

<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<ipython-input-34-36acebcf87a6>:2: DeprecationWarning: invalid escape sequence \S
  with open("C:\\Users\\jagwa\\Google Drive\\courses\\at UCD\\STA\STA 208 ML\\project\\corpus2.csv") as f:


['entertainment']
['allow', 'experience', 'situation', 'otherwise', 'would', 'never', 'opportunity', 'experience', 'allow', 'range', 'emotion', 'short', 'period', 'without', 'emotion', 'hurting', 'movie', 'bring', 'richness', 'variety', 'depth', 'screen']
['think', 'provide', 'decent', 'entertainment', 'movie', 'general', 'science', 'fiction']
[]
['offer', 'chance', 'escape', 'forget', 'worry', 'responsibility', 'hours']
['give', 'happiness']
['help', 'relax', 'forget', 'problem', 'inspire']
['movie', 'awarnes', 'socity', 'movie.all', 'people', 'please', 'watch', 'movie']


In [35]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data_2)
corpus = [dictionary.doc2bow(text) for text in text_data_2]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [36]:
NUM_TOPICS = 3
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model3_2.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.081*"movie" + 0.046*"socity" + 0.046*"movie.all" + 0.046*"awarnes"')
(1, '0.052*"emotion" + 0.052*"allow" + 0.052*"experience" + 0.052*"movie"')
(2, '0.056*"forget" + 0.055*"offer" + 0.055*"responsibility" + 0.055*"worry"')


In [37]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3_2.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.061*"forget" + 0.061*"help" + 0.061*"inspire" + 0.061*"relax"')
(1, '0.074*"movie" + 0.052*"experience" + 0.052*"allow" + 0.052*"emotion"')
(2, '0.075*"entertainment" + 0.043*"think" + 0.043*"science" + 0.043*"general"')


In [39]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model3_2.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\jagwa\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


From the results above, it can be seen that the three topics extracted from the reponses are about the equal size. The first topic can be recognized as hedonic experience, as the top words in the topic are "reduce stress" and "improve happiness". The second topic can be seen as social experiecne, as there are words like "people", "discuss idea" in the second topic. The third topic can be treated as eudaimonic experience, as there are words like "insight", and "problem" in the topic. The differences and distinction between these three topics are not very obvious, probably due to the relative small sample size.

# Discussion and Summary
The present project utilizes the LDA method to solicit topics from two corpuses. The first part uses LDA method to elicit topics from textual responses. The second and third part uses LDA method and improves the topic modeling on two different textual responses. The results showed that LDA has great potential in recognizing and differntiating different topics. This technique can be applied in various social scientific studies in the future and help scholars understand textual responses better.

## Results
The topic modeling results have significant implications for entertaiment studies. The first topic modeling shows that two topics are prominent when people answering the question about video game function in families. People genreally believe that video games help families unite and they get a lot of fun through video games. However, when people answer the same question regarding movies, they also mention insights, inspirations, and other eudaimonic functions of movies. It seems movies have more functions than video games but when we looked deeper at the reponses about video games, we also see responses like "learning" and "coorperation" in the corpus. The eudaimonic topic is not prominent but still exists. So far, it is probably safe to say that ,in genreal, entertainment has roughly three functions in people's life:  
1) Hedonic, people look for fun in entertainment, and seek escapism, diversion, and relaxation.  
2) Eudaimonism, people seek meanings and personal grwoth in entertainment. They learn to grow and look for inspirations and insights in entertainment contents.  
3) Social, people see entertainment as a social venue and look for relatedness with other people when consuming entertainment content.  

## Limitations
The present project is not without limitations.  
First, the results of the topic modeling are not stable. Different iterations usually provides different resutls, and an ideal result needs multiple trials. It is probably due to the small descrepancies between different topics and responses.  
Second, the differences between topics are not very obvious. It still needs very careful human inspection, or even expert inspection to understand and label the different topics. It is probably because it is unguided machine learning, there are no guidelines on how the machine can learn the topics, but also due to the small sample size. If we can increase the sample size to about 1000, I believe the results can be a lot more stable and differentiable.  

## Summary
Overall, the present project utilizes machine learnign strategy to traditional social scientific studies. Using natural human textual responses, with the help of LDA machine learning, researchers can achieve a lot more than traditional human coding. The result of three topics can inspire a lot more other studies in this area and future research can also utilize this machine learning technique to understand other textual responses in other social scientific areas of interest.